Can we predict the severity of an accident based on time, weather, and location data?
    Using Random Forest, Logistic Regression, and SVM (multiclass classification)

In [2]:
#imports
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

###Random Forest

In [3]:
#Random Forest

#dataset
df = pd.read_csv('../Data/us_accidents_sample_500k_clean.csv')

#target 
#df.Severity.value_counts() #already int

# Extract time features from Start_Time
df['Start_Time'] = pd.to_datetime(df['Start_Time'])
df['Hour'] = df['Start_Time'].dt.hour
df['Day_of_Week'] = df['Start_Time'].dt.dayofweek
df['Month'] = df['Start_Time'].dt.month

# Reduce high-cardinality by grouping rare categories for 'State' and 'Weather_Simple'
state_counts = df['State'].value_counts()
rare_states = state_counts[state_counts < 500].index  # threshold: group states with <500 occurrences
df['State'] = df['State'].replace(rare_states, 'Other_State')

weather_counts = df['Weather_Simple'].value_counts()
rare_weather = weather_counts[weather_counts < 1000].index
df['Weather_Simple'] = df['Weather_Simple'].replace(rare_weather, 'Other_Weather')

# Small stratified subsample for fast experiments (optional)
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1, train_size=0.2, random_state=42)
train_idx, _ = next(sss.split(df, df['Severity']))
df_sample = df.iloc[train_idx].copy()
print('Subsample size for experiments:', len(df_sample))

# show columns
df.columns

Subsample size for experiments: 90374


Index(['ID', 'Severity', 'Start_Time', 'End_Time', 'Lat', 'Lng', 'Street',
       'City', 'County', 'State', 'Wind_Chill(F)', 'Humidity(%)',
       'Pressure(in)', 'Sunrise_Sunset', 'Severity_Label', 'Is_Day',
       'Temperature(F)', 'Visibility(mi)', 'Wind_Speed(mph)',
       'Precipitation(in)', 'Weather_Condition', 'Weather_Simple', 'Hour',
       'Day_of_Week', 'Month'],
      dtype='object')

In [4]:
# Stratified subsample for faster experiments (20% of data, preserves class balance)
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1, train_size=0.2, random_state=42)
sample_idx, _ = next(sss.split(df, df['Severity']))
df_sample = df.iloc[sample_idx].copy()
print('Stratified subsample created: df_sample with', len(df_sample), 'rows')


Stratified subsample created: df_sample with 90374 rows


In [5]:
#split into training and testing
train, test = train_test_split(
    df,
    test_size=0.20,
    random_state=42,
    stratify=df.Severity  #4 classes balanced
)

In [6]:
#chose feature + target
features = ['Weather_Simple', 'Visibility(mi)', 'Sunrise_Sunset', 'State', 
            'Hour', 'Day_of_Week', 'Month', 'Precipitation(in)', 'Humidity(%)', 'Wind_Speed(mph)']
target = 'Severity'

# check null values for new features
print("Null values per feature:")
for col in features:
    print(f"{col}: {df[col].isnull().sum()}")

# create stratified train/test split
train, test = train_test_split(
    df,
    test_size=0.20,
    random_state=42,
    stratify=df.Severity  #4 classes balanced
)

# For ML: drop rows with missing values in features (after split to avoid leakage)
train = train.dropna(subset=features + ['Severity'])
test = test.dropna(subset=features + ['Severity'])
print(f"\nTraining set size after dropping NAs: {len(train)}")
print(f"Test set size after dropping NAs: {len(test)}")

Null values per feature:
Weather_Simple: 0
Visibility(mi): 10185
Sunrise_Sunset: 1166
State: 0
Hour: 0
Day_of_Week: 0
Month: 0
Precipitation(in): 0
Humidity(%): 10016
Wind_Speed(mph): 0

Training set size after dropping NAs: 350888
Test set size after dropping NAs: 87662

Training set size after dropping NAs: 350888
Test set size after dropping NAs: 87662


In [7]:
#prepare training and testing subsets
X_train = train[features]
y_train = train[target]

X_test = test[features]
y_test = test[target]

In [8]:
#preprocessing(OneHot and num combo)

# numeric pipeline: impute then scale
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

num_features = ['Visibility(mi)', 'Precipitation(in)', 'Humidity(%)', 'Wind_Speed(mph)', 'Hour', 'Day_of_Week', 'Month']
cat_features = ['Weather_Simple', 'Sunrise_Sunset', 'State']

preprocess = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_features),
        ('num', numeric_transformer, num_features)
    ],
    remainder='drop'
)

In [9]:
#pipeline
rand_forest = Pipeline(steps=[
    ('preprocess', preprocess),
    ('model', RandomForestClassifier(n_estimators=100, max_depth=12, max_features='sqrt', n_jobs=-1, random_state=42))
])

In [10]:
#cross-validation only for Training
scores = cross_val_score(rand_forest, X_train, y_train, cv=10, scoring='accuracy', n_jobs=-1)
print("Random Forest Cross-Validation Score:", scores)
print("Random Forest Mean Cross-Validation accuracy:", scores.mean())


KeyboardInterrupt: 

###logistic regression

In [32]:
#logistic regression multiclass 
#split data 
y = df[target] #target
X = df[features] #Features
#train and test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y           #stratify by the target == severity
)

print(f"Training set size: {len(X_train)}")
print(f"Test set size: {len(X_test)}")

Training set size: 361499
Test set size: 90375


In [33]:

#drop rows with NAs
train_data = pd.concat([X_train, y_train], axis=1).dropna(subset=features + [target])
X_train = train_data[features]
y_train = train_data[target]

test_data = pd.concat([X_test, y_test], axis=1).dropna(subset=features + [target])
X_test = test_data[features]
y_test = test_data[target]

In [34]:
#preprocessing
# reuse numeric_transformer defined earlier
cat_cols = ['Weather_Simple', 'Sunrise_Sunset', 'State']
num_cols = ['Visibility(mi)', 'Precipitation(in)', 'Humidity(%)', 'Wind_Speed(mph)', 'Hour', 'Day_of_Week', 'Month']

preprocess = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_features),
        ('num', numeric_transformer, num_cols)
    ],
    remainder='drop'
)

In [35]:
#pipeline
log_model = Pipeline(steps=[
    ('preprocess', preprocess),
    ('model', LogisticRegression(max_iter=1000))
])

In [36]:
#cross validation
scores = cross_val_score(
    log_model, X_train, y_train, cv=10, scoring='accuracy', n_jobs=-1
)

print("logistic regression Cross-Validation scores:", scores)
print("logistic regression Mean Cross-Validation accuracy:", scores.mean())

logistic regression Cross-Validation scores: [0.77725213 0.77762262 0.77765112 0.77739463 0.77745162 0.77710964
 0.77736613 0.77779361 0.77755928 0.77684679]
logistic regression Mean Cross-Validation accuracy: 0.7774047547837349


###XGBOOST

In [39]:
import xgboost as xgb
#split training and testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
#EDA preprocessing reusing 
cat_cols = ['Weather_Simple', 'Sunrise_Sunset', 'State']
num_cols = ['Visibility(mi)', 'Precipitation(in)', 'Humidity(%)', 'Wind_Speed(mph)', 'Hour', 'Day_of_Week', 'Month']

preprocess = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_features),
        ('num', numeric_transformer, num_cols)
    ],
    remainder='drop'
)

In [40]:
#label encoding 
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder() #encode labels as ints
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)

In [47]:
#pipeline
xgb_model = xgb.XGBClassifier(
    objective='multi:softprob', #multi class classificaiton
    eval_metric='mlogloss',    #evaluatoin metric
    n_jobs=-1,
    random_state=42
)

xgb_pipeline = Pipeline(steps=[
    ('preprocess', preprocess),
    ('model', xgb_model)
])

In [48]:
#cross validation
scores_xgb = cross_val_score(
    xgb_pipeline,
    X_train,
    y_train_encoded,
    cv=10,
    scoring='accuracy',
    n_jobs=-1    
)
print("XGBoost Cross-Validation scores:", scores_xgb)
print("Mean Cross-Validation accuracy:", scores_xgb.mean())

XGBoost Cross-Validation scores: [0.78143845 0.78251729 0.78141079 0.7819917  0.78204703 0.78121715
 0.78052559 0.78143845 0.7806639  0.78198567]
Mean Cross-Validation accuracy: 0.7815236016201118


In [11]:
#XGBOOST model 
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder() #encode labels as ints
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)

xgb_pipeline = Pipeline(steps=[
    ('preprocess', preprocess),
    ('model', xgb.XGBClassifier(eval_metric='mlogloss', n_jobs=-1, random_state=42))
])

scores_xgb = cross_val_score(xgb_pipeline, X_train, y_train_encoded, cv=3, scoring='accuracy', n_jobs=-1)
print("XGBoost Cross-Validation scores:", scores_xgb)
print("Mean XGBoost Cross-Validation accuracy:", scores_xgb.mean())

XGBoost Cross-Validation scores: [0.78128981 0.78146935 0.78152733]
Mean XGBoost Cross-Validation accuracy: 0.7814288322726949


###Best Model: 

In [13]:
#testing the best ML model
from sklearn.model_selection import GridSearchCV
#hyperparam tuning
param_grid = {
    'model__n_estimators': [100, 200],
    'model__max_depth': [3, 5],
    'model__learning_rate': [0.1],
    'model__subsample': [0.8, 1.0]
}

#gridsearch
grid_search = GridSearchCV(
    estimator=xgb_pipeline, 
    param_grid=param_grid, 
    cv=5, 
    scoring='accuracy', 
    n_jobs=-1,
    verbose=1
)
grid_search.fit(X_train, y_train_encoded)
print("Best hyperparameters:", grid_search.best_params_)
print("Best CV accuracy:", grid_search.best_score_)
#refit final model with best hyperparameter
final_model = grid_search.best_estimator_
final_model.fit(X_train, y_train_encoded)

#final Eval
# Predict on test set
y_pred = final_model.predict(X_test)

# Accuracy and classification report
accuracy = accuracy_score(y_test_encoded, y_pred)
print("Test set accuracy:", accuracy)
print("Classification report:\n", classification_report(
    y_test_encoded, 
    y_pred, 
    target_names=[str(c) for c in le.classes_],  # convert to strings
    zero_division=0  
))

Fitting 5 folds for each of 8 candidates, totalling 40 fits
Best hyperparameters: {'model__learning_rate': 0.1, 'model__max_depth': 5, 'model__n_estimators': 200, 'model__subsample': 0.8}
Best CV accuracy: 0.7808759482466217
Best hyperparameters: {'model__learning_rate': 0.1, 'model__max_depth': 5, 'model__n_estimators': 200, 'model__subsample': 0.8}
Best CV accuracy: 0.7808759482466217
Test set accuracy: 0.7816727886655563
Classification report:
               precision    recall  f1-score   support

           1       0.00      0.00      0.00       857
           2       0.79      0.99      0.88     68128
           3       0.58      0.06      0.11     16413
           4       0.62      0.01      0.01      2264

    accuracy                           0.78     87662
   macro avg       0.50      0.26      0.25     87662
weighted avg       0.74      0.78      0.70     87662

Test set accuracy: 0.7816727886655563
Classification report:
               precision    recall  f1-score   suppo